In [195]:
import random
import copy
import numpy as np
import pandas as pd

## 交叉

### 部分匹配交叉PMX

In [319]:
def PMX(parent1,parent2):
    geneNum = len(parent1) # 基因数量（决策变量数量）
    parent1_copy=copy.deepcopy(parent1) # 子代1
    parent2_copy=copy.deepcopy(parent2) # 子代2
    # 选择交叉点
    point1 = np.random.randint(0, geneNum)
    point2 = np.random.randint(0, geneNum)
    while point1 > point2 or point1 == point2:
        point1 = np.random.randint(0, geneNum)
        point2 = np.random.randint(0, geneNum)


    # 记录交叉项
    fragment1 = parent1[point1:point2 + 1]
    fragment2 = parent2[point1:point2 + 1]

    # 交叉
    parent1_copy[point1:point2+1], parent2_copy[point1:point2+1] = fragment2,fragment1
    
        # 头部修正
    if point1 != 0:

        for idx in range(point1):
            # 子代1
            while parent1_copy[idx] in fragment2:
                duplicate_idx = list(fragment2).index(parent1_copy[idx])  # 找到重复位置的索引
                modify_value = fragment1[duplicate_idx]  # 修正后的值
                parent1_copy[idx] = modify_value
            # 子代2
            while parent2_copy[idx] in fragment1:
                duplicate_idx = list(fragment1).index(parent2_copy[idx])  # 找到重复位置的索引
                modify_value = fragment2[duplicate_idx]  # 修正后的值
                parent2_copy[idx] = modify_value

    # 尾部修正
    if point2 != geneNum-1:
        for idx in range(point2+1,geneNum):
            # 子代1
            while parent1_copy[idx] in fragment2:
                duplicate_idx = list(fragment2).index(parent1_copy[idx])  # 找到重复位置的索引
                modify_value = fragment1[duplicate_idx]  # 修正后的值
                parent1_copy[idx] = modify_value
            # 子代2
            while parent2_copy[idx] in fragment1:
                duplicate_idx = list(fragment1).index(parent2_copy[idx])  # 找到重复位置的索引
                modify_value = fragment2[duplicate_idx]  # 修正后的值
                parent2_copy[idx] = modify_value
    return parent1_copy,parent2_copy

### 顺序交叉

In [320]:
def OX(parent1,parent2):
    geneNum = len(parent1) # 基因数量（决策变量数量）
    parent1_copy=copy.deepcopy(parent1) # 子代1
    parent2_copy=copy.deepcopy(parent2) # 子代2
    # 选择交叉点
    point1 = np.random.randint(0, geneNum)
    point2 = np.random.randint(0, geneNum)
    while point1 > point2 or point1 == point2:
        point1 = np.random.randint(0, geneNum)
        point2 = np.random.randint(0, geneNum)
    # 记录中间段
    fragment1 = parent1[point1:point2 + 1]
    fragment2 = parent2[point1:point2 + 1]
    # 获取其余项顺序序列
    if point2 != geneNum - 1:
        order1 = list(parent1[point2 + 1:]) + list(parent1[:point2+1])
        order2 = list(parent2[point2 + 1:]) + list(parent2[:point2+1])
    else:
        order1 = copy.deepcopy(parent1)
        order2 = copy.deepcopy(parent2)

    # 删除重复项
    del_order1 = [i for i in order1 if i not in fragment2]
    del_order2 = [i for i in order2 if i not in fragment1]
    # 顺序复制
    parent1_copy[point2+1:]=del_order2[:len(parent1_copy[point2+1:])]
    parent1_copy[:point1]=del_order2[len(parent1_copy[point2+1:]):]

    parent2_copy[point2+1:]=del_order1[:len(parent1_copy[point2+1:])]
    parent2_copy[:point1]=del_order1[len(parent1_copy[point2+1:]):]
    return parent1_copy,parent2_copy

### 循环交叉

In [321]:
def CX(parent1,parent2):
    geneNum = len(parent1) # 基因数量（决策变量数量）
    parent1_copy = copy.deepcopy(parent1)  # 子代1
    parent2_copy = copy.deepcopy(parent2)  # 子代2
    point=np.random.randint(0, geneNum) # 第一个交叉点
    parent1_copy[point],parent2_copy[point]=parent2_copy[point],parent1_copy[point]
    while len(np.unique(parent2_copy))!=geneNum:
        duplicate=np.sum(parent2_copy)-np.sum(np.unique(parent2_copy)) #找到重复的元素
        duplicate_idx = np.where(parent2_copy==duplicate)[0] 
        point=duplicate_idx[np.where(duplicate_idx!=point)[0][0]]# 找到下一个交换点的索引
        parent1_copy[point],parent2_copy[point]=parent2_copy[point],parent1_copy[point]
    return parent1,parent2

### 交叉函数

In [111]:
def crossover(parent1, parent2, method="PMX"):
    if method == "PMX":
        return PMX(parent1, parent2)
    elif method == "OX":
        return OX(parent1, parent2)
    elif method == "CX":
        return CX(parent1, parent2)

## 变异

### 相反变异

In [188]:
def inversion(parent):
    """
    相反变异
    """
    geneNum=len(parent)
    # 选择交叉点
    point1 = np.random.randint(0, geneNum)
    point2 = np.random.randint(0, geneNum)
    while point1 > point2 or point1 == point2:
        point1 = np.random.randint(0, geneNum)
        point2 = np.random.randint(0, geneNum)
    # 逆序
    parent[point1:point2+1]=parent[point1:point2+1][::-1]
    return parent

In [307]:
def mutation(parent,method="inversion"):
    """
    变异
    """
    if method=="inversion":
        return inversion(parent)

In [334]:
def fitness(parent):
    """
    适应度函数
    """
    dist = 0
    for i in range(len(parent)):
        if i == len(parent) - 1:
            dist = dist + distance[parent[i], parent[0]]
        else:
            dist = dist + distance[parent[i], parent[i + 1]]
    return 1/dist

In [314]:
def get_best(group):
    dist = np.array([fitness(i) for i in group]) # 个体适应度
    return np.max(dist),group[np.argmax(dist)]

In [288]:
def select(group):
    """
    个体筛选
    """
    dist = np.array([fitness(i) for i in group]) # 个体适应度
    probability = dist / dist.sum() # 选择概率
    cum_prob = np.cumsum(probability) # 累计概率
    dart = np.random.random(len(group))
    # while
    new_group = []
    selected_parent_idx = []  # 记录已经被选中的个体索引值
    for i in range(len(group)):
        selected_idx = ((dart[i] - cum_prob) > 0).sum() # 被选中的个体的索引
        if selected_idx not in selected_parent_idx:
            new_group.append(list(group[selected_idx]))
        selected_parent_idx.append(selected_idx)
    return new_group

In [309]:
def group_crossover(group, method="PMX"):
    """
    对群体进行交叉操作
    """
    new_crossover_group = copy.deepcopy(group)
    for i in range(int(len(group) / 2)):
        sample_two = random.sample(group, 2)  # 任取两个出来交叉
        parent1 = sample_two[0]
        parent2 = sample_two[1]
        group.remove(parent1)
        group.remove(parent2)
        p1, p2 = crossover(np.array(parent1), np.array(parent2), method=method)
        new_crossover_group.append(p1)
        new_crossover_group.append(p2)
        
    return new_crossover_group

In [310]:
def group_mutation(group):
    """
    种群变异
    """
    new_mutation_group = []
    for i in range(len(group)):
        if random.random() < mutation_rate:
            parent = mutation(group[i])
            new_mutation_group.append(parent)
        else:
            new_mutation_group.append(group[i])
    return new_mutation_group

## 正式训练

### 参数及变量准备

In [332]:
group = []  # 种群
init_num = 100  # 初始化种群数量
iterations = 500  # 迭代次数
mutation_rate = 0.3  # 变异概率
position = pd.read_excel("position.xlsx", engine="openpyxl")  #读取位置信息
w = len(position)  # 坐标点数量
distance = np.zeros((w, w))  # 距离矩阵

# 填充距离矩阵
for i in range(w):
    for j in range(w):
        if i != j:
            distance[i,j] = np.linalg.norm(position.iloc[i,:]-position.iloc[j,:])

# 初始化种群
for i in range(init_num):
    p = np.random.choice(range(w), w, replace=False)
    group.append(list(p))

### 开始迭代

In [335]:
for i in range(iterations):
    # 自然选择
    new_group = select(group)
    # 交叉
    new_group = group_crossover(new_group)
    # 变异
    new_group = group_mutation(new_group)
    best_fit,best_sol = get_best(new_group)
    print(best_fit)

116
125
121
133
137
121
121
136
120
128
128
124
132
117
120
128
121
129
116
129
132
120
132
105
121
124
124
121
128
133
136
117
125
129
144
116
128
117
132
125
120
121
128
112
132
120
120
121
120
116
129
125
129
133
132
129
120
125
124
133
129
128
120
125
121
136
132
124
140
132
129
124
113
137
136
132
128
129
121
125
125
124
128
128
129
117
128
113
125
128
128
120
128
133
116
132
125
129
136
125
140
128
128
113
128
117
133
120
128
121
120
128
117
136
125
125
121
117
125
132
129
121
121
129
125
124
121
125
125
112
120
121
128
133
133
129
120
117
132
124
121
129
113
133
105
136
128
121
128
125
129
116
124
121
121
129
124
116
124
117
132
129
125
125
128
125
113
121
133
128
129
121
125
109
121
125
136
128
121
129
128
129
125
125
136
116
129
128
125
129
112
128
120
129
125
128
124
113
132
133
121
136
124
112
128
124
121
132
117
117
129
133
132
128
120
132
132
120
136
125
129
124
129
124
136
120
128
124
133
128
136
133
129
116
128
137
137
128
128
132
132
121
120
133
120
133
136
136
124
129
